In [1]:
import numpy as np
import torch
from torchvision import models, transforms, datasets
import torch.nn.functional as F
from torch import nn, optim
import face_detector
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model_1 = models.resnet18(pretrained=True)

In [4]:
class Network(nn.Module):
    
    '''Predicts the probabilty that the player playing white is going to win.'''
    
    def __init__(self):
        super(Network, self).__init__()
        
        
        self.layer1 = nn.Linear(224,100)
        self.layer2 = nn.Linear(100, 50)
        self.layer3 = nn.Linear(50, 2)

    def forward(self, x):
        
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x))
        
        return x

In [5]:
# for param in model_1.parameters():
#     param.requires_grad = False

model_1.classifier = Network()

In [6]:
data_transforms = transforms.Compose([face_detector.FaceCropper(),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])

In [7]:
def random_frame_selector(video_source):
    
    video = cv2.VideoCapture(video_source)

    video_length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    random_frame = np.random.randint(0, video_length)

    video.set(1, random_frame)

    _, frame = video.read()
    
    return frame

In [8]:
extensions = '.mp4'

data_dir = './videos'

train_dir = data_dir + '/train_sample_videos'
test_dir = data_dir + '/test_videos'

In [9]:
train_data = datasets.DatasetFolder(train_dir,
                                    loader=random_frame_selector,
                                    extensions=extensions,
                                    transform=data_transforms)

In [10]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

In [11]:
train_data

Dataset DatasetFolder
    Number of datapoints: 400
    Root location: ./videos/train_sample_videos
    StandardTransform
Transform: Compose(
               CenterCrop(size=(224, 224))
               ToTensor()
           )

In [12]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model_1.classifier.parameters(), lr=0.001)
model_1.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
epoch = 1

for e in range(epoch):
    train_losses = []
    running_loss = 0
    model_1.train()
    for images, labels in trainloader:
        print(labels)
        images = images.to(device)
        labels = labels.to(device)

        log_ps = model_1.forward(images)
        loss = criterion(log_ps, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_losses.append(running_loss/len(trainloader))

    print(f"Training loss: {running_loss}")

tensor([1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
        2, 2, 1, 2, 2, 2, 2, 1])
tensor([2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2,
        1, 2, 2, 2, 1, 2, 2, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1,
        2, 2, 2, 2, 2, 2, 1, 2])
tensor([2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2,
        2, 1, 2, 1, 2, 2, 2, 2])
tensor([2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2,
        2, 1, 2,